In [1]:
from transformers import pipeline
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [2]:
import re
import pandas as pd

beyonce_df = pd.read_csv('/Users/mertakcay/Projects/Knowledge Graphs/Neo4j/context.csv')
context_list = beyonce_df['context'].apply(lambda x : re.sub(r'[^\w\s]', '', x))

In [3]:
triplets = list()
for context in context_list:
    triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(context,return_tensors=True, return_text=False)[0]["generated_token_ids"]])
    extracted_triplets = extract_triplets(extracted_text[0])
    print(extracted_triplets)
    triplets.append(extracted_triplets)

[{'head': 'Destinys Child Managed', 'type': 'has part', 'tail': 'Mathew Knowles'}, {'head': 'Dangerously in Love 2003', 'type': 'publication date', 'tail': '2003'}, {'head': 'Crazy in Love', 'type': 'performer', 'tail': 'Destinys Child Managed'}, {'head': 'Crazy in Love', 'type': 'part of', 'tail': 'Dangerously in Love 2003'}, {'head': 'Baby Boy', 'type': 'performer', 'tail': 'Destinys Child Managed'}, {'head': 'Baby Boy', 'type': 'part of', 'tail': 'Dangerously in Love 2003'}]
[{'head': 'Déjà Vu Irreplaceable', 'type': 'part of', 'tail': 'BDay 2006'}, {'head': 'Déjà Vu Irreplaceable', 'type': 'followed by', 'tail': 'Beautiful Liar'}, {'head': 'Beautiful Liar', 'type': 'part of', 'tail': 'BDay 2006'}, {'head': 'Beautiful Liar', 'type': 'follows', 'tail': 'Déjà Vu Irreplaceable'}, {'head': 'Dreamgirls 2006', 'type': 'publication date', 'tail': '2006'}, {'head': 'Dreamgirls 2006', 'type': 'publication date', 'tail': '2006'}, {'head': 'Dreamgirls 2006', 'type': 'publication date', 'tail':